In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/New Approach/models') # append the directory containing the module

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import EncoderEEG
import deepcnn
import cnn2dlstm
import cnn1dlstm
import traintest

In [ ]:
import importlib
importlib.reload(traintest) # reload the module

<module 'traintest' from '/content/drive/MyDrive/New Approach/models/traintest.py'>

In [ ]:
torch.cuda.is_available()
if torch.cuda.is_available():
    print("GPU Name:", torch.cuda.get_device_name(0))

GPU Name: Tesla T4


In [ ]:
dataset=torch.load(r"/content/drive/MyDrive/Data/eeg_55_95_std.pth")
labels=dataset['labels']
print(labels)

<ipython-input-4-d39b2ac6a785>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  dataset=torch.load(r"/content/drive/MyDrive/Data/eeg_55_95_std.pth")


['n02389026', 'n03888257', 'n03584829', 'n02607072', 'n03297495', 'n03063599', 'n03792782', 'n04086273', 'n02510455', 'n11939491', 'n02951358', 'n02281787', 'n02106662', 'n04120489', 'n03590841', 'n02992529', 'n03445777', 'n03180011', 'n02906734', 'n07873807', 'n03773504', 'n02492035', 'n03982430', 'n03709823', 'n03100240', 'n03376595', 'n03877472', 'n03775071', 'n03272010', 'n04069434', 'n03452741', 'n03792972', 'n07753592', 'n13054560', 'n03197337', 'n02504458', 'n02690373', 'n03272562', 'n04044716', 'n02124075']


In [ ]:
data=dataset['dataset']

In [ ]:
splits=torch.load(r"/content/drive/MyDrive/Data/block_splits_by_image_all.pth")

<ipython-input-6-02f78271f0ac>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  splits=torch.load(r"/content/drive/MyDrive/Data/block_splits_by_image_all.pth")


In [ ]:
x_train,x_val,x_test,y_train,y_val,y_test=traintest.Splitter(dataset,splits,subject_no=0)

In [ ]:
# For using 2D CNN or similar models, we need to add an extra dimension to our data for which we can use unsqueeze function

x_train=torch.stack(x_train)
#x_train=x_train.unsqueeze(3)
x_val=torch.stack(x_val)
#x_val=x_val.unsqueeze(3)
x_test=torch.stack(x_test)
#x_test=x_test.unsqueeze(3)
y_train=torch.tensor(y_train)
y_val=torch.tensor(y_val)
y_test=torch.tensor(y_test)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
num_classes = 40
learning_rate = 0.005
num_epochs = 50

model = EncoderEEG.EEGEncoder(num_classes=40).to(device)
print(model)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

EEGEncoder(
  (temporal_block): TemporalBlock(
    (network): Sequential(
      (0): Conv1d(128, 64, kernel_size=(3,), stride=(1,), padding=(3,))
      (1): ReLU()
      (2): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(6,), dilation=(2,))
      (3): ReLU()
      (4): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(12,), dilation=(4,))
      (5): ReLU()
      (6): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(24,), dilation=(8,))
      (7): ReLU()
    )
  )
  (spatial_block): SpatialBlock(
    (network): Sequential(
      (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU()
    )
  )
  (res_blocks): ModuleList(
    (0-1): 2 x ResidualBlock(
      (conv1)

In [ ]:
from torch.utils.data import DataLoader, TensorDataset

train_dataset = TensorDataset(x_train, y_train)
val_dataset = TensorDataset(x_val,y_val)
test_dataset = TensorDataset(x_test,y_test)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset,batch_size=16,shuffle=True)
test_loader = DataLoader(test_dataset,batch_size=16,shuffle=True)

In [ ]:
#for inputs,labels in train_loader:
 #   print(inputs.shape)

In [ ]:
# Call training function
traintest.train_model(model, train_loader, val_loader,criterion,optimizer,device,num_epochs)

Epoch 1/50, Training Loss: 3.6681, Validation Loss: 3.6328, Training Accuracy: 3.14%, Validation Accuracy: 2.40%
Epoch 2/50, Training Loss: 3.5224, Validation Loss: 3.4731, Training Accuracy: 4.09%, Validation Accuracy: 4.30%
Epoch 3/50, Training Loss: 3.2692, Validation Loss: 3.1000, Training Accuracy: 5.50%, Validation Accuracy: 7.56%
Epoch 4/50, Training Loss: 2.9927, Validation Loss: 3.1904, Training Accuracy: 7.69%, Validation Accuracy: 6.16%
Epoch 5/50, Training Loss: 2.7882, Validation Loss: 2.6539, Training Accuracy: 9.79%, Validation Accuracy: 12.06%
Epoch 6/50, Training Loss: 2.6276, Validation Loss: 2.5618, Training Accuracy: 11.04%, Validation Accuracy: 11.21%
Epoch 7/50, Training Loss: 2.5078, Validation Loss: 2.5470, Training Accuracy: 13.96%, Validation Accuracy: 14.66%
Epoch 8/50, Training Loss: 2.4525, Validation Loss: 2.3131, Training Accuracy: 13.96%, Validation Accuracy: 15.82%
Epoch 9/50, Training Loss: 2.3989, Validation Loss: 2.3749, Training Accuracy: 14.69%, Va

In [ ]:
traintest.evaluate_model(model, test_loader)

Test Accuracy: 40.16%


In [ ]:
#Saving the model

torch.save(model.state_dict(), 'EEGEncoder_state_dict.pth')  # Save only the state dict
torch.save(model, 'EEGEncoder.pth')  # Save the entire model
torch.save(optimizer.state_dict(), 'optimizer_state_dict.pth')

##Further Training to be done on a new segment of our dataset

In [ ]:
dataset=torch.load(r"/content/drive/MyDrive/Data/eeg_14_70_std.pth")
labels=dataset['labels']
dataset.keys()

<ipython-input-9-909b9687f119>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  dataset=torch.load(r"/content/drive/MyDrive/Data/eeg_14_70_std.pth")


dict_keys(['dataset', 'labels', 'images'])

In [ ]:
data=dataset['dataset']

In [ ]:
splits=torch.load(r"/content/drive/MyDrive/Data/block_splits_by_image_all.pth")

<ipython-input-10-02f78271f0ac>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  splits=torch.load(r"/content/drive/MyDrive/Data/block_splits_by_image_all.pth")


In [ ]:
x_train,x_val,x_test,y_train,y_val,y_test=traintest.Splitter(dataset,splits,subject_no=0)

In [ ]:
x_train=torch.stack(x_train)
#x_train=x_train.unsqueeze(3)
x_val=torch.stack(x_val)
#x_val=x_val.unsqueeze(3)
x_test=torch.stack(x_test)
#x_test=x_test.unsqueeze(3)
y_train=torch.tensor(y_train)
y_val=torch.tensor(y_val)
y_test=torch.tensor(y_test)

In [ ]:
from torch.utils.data import DataLoader, TensorDataset

train_dataset = TensorDataset(x_train, y_train)
val_dataset = TensorDataset(x_val,y_val)
test_dataset = TensorDataset(x_test,y_test)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset,batch_size=16,shuffle=True)
test_loader = DataLoader(test_dataset,batch_size=16,shuffle=True)

In [ ]:
model = torch.load('/content/drive/MyDrive/New Approach/EEGEncoder.pth')
model.load_state_dict(torch.load('/content/drive/MyDrive/New Approach/EEGEncoder_state_dict.pth'))
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
optimizer.load_state_dict(torch.load('/content/drive/MyDrive/New Approach/optimizer_state_dict.pth'))

<ipython-input-17-148cb984ddee>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load('/content/drive/MyDrive/New Approach/EEGEncoder.pth')
<ipython-input-17-1

In [ ]:
traintest.train_model(model, train_loader, val_loader,criterion,optimizer,num_epochs)

Epoch 1/50, Training Loss: 2.0205, Validation Loss: 1.9329, Training Accuracy: 25.53%, Validation Accuracy: 25.88%
Epoch 2/50, Training Loss: 1.8511, Validation Loss: 1.8485, Training Accuracy: 29.55%, Validation Accuracy: 29.68%
Epoch 3/50, Training Loss: 1.7358, Validation Loss: 1.7577, Training Accuracy: 32.18%, Validation Accuracy: 31.68%
Epoch 4/50, Training Loss: 1.6776, Validation Loss: 1.8013, Training Accuracy: 34.20%, Validation Accuracy: 27.88%
Epoch 5/50, Training Loss: 1.6362, Validation Loss: 1.7587, Training Accuracy: 35.40%, Validation Accuracy: 29.98%
Epoch 6/50, Training Loss: 1.5866, Validation Loss: 1.6811, Training Accuracy: 35.76%, Validation Accuracy: 34.13%
Epoch 7/50, Training Loss: 1.5709, Validation Loss: 1.7480, Training Accuracy: 36.30%, Validation Accuracy: 33.48%
Epoch 8/50, Training Loss: 1.5314, Validation Loss: 1.6965, Training Accuracy: 37.69%, Validation Accuracy: 32.63%
Epoch 9/50, Training Loss: 1.5070, Validation Loss: 1.6625, Training Accuracy: 3

In [ ]:
traintest.evaluate_model(model, test_loader)

Test Accuracy: 36.50%


In [ ]:
#Saving the model

torch.save(model.state_dict(), 'EEGEncoder_state_dict2.pth')  # Save only the state dict
torch.save(model, 'EEGEncoder2.pth')  # Save the entire model
torch.save(optimizer.state_dict(), 'optimizer_state_dict2.pth')

## Not Relevant

In [ ]:
sample_input = torch.randn(16, 128, 440)  # Create a sample input tensor

# Initialize your model
num_classes = 40  # Adjust based on the number of classes in your task
model =EncoderEEG.EEGEncoder(num_classes)  # Use the correct model name here

# Pass the sample input through the model
try:
    output = model(sample_input)
    print("Output shape:", output.shape)  # Print the output shape
except Exception as e:
    print("An error occurred:", e)

An error occurred: Given groups=1, weight of size [64, 40, 3], expected input[16, 128, 440] to have 40 channels, but got 128 channels instead


In [ ]:
print(model.parameters)

<bound method Module.parameters of EEGEncoder(
  (temporal_block): TemporalBlock(
    (network): Sequential(
      (0): Conv1d(128, 64, kernel_size=(3,), stride=(1,), padding=(3,))
      (1): ReLU()
      (2): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(6,), dilation=(2,))
      (3): ReLU()
      (4): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(12,), dilation=(4,))
      (5): ReLU()
      (6): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(24,), dilation=(8,))
      (7): ReLU()
    )
  )
  (spatial_block): SpatialBlock(
    (network): Sequential(
      (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU()
    )
  )
  (res_blocks): ModuleList(
    (0-1